<h1>2023 California Medical Board Analysis</h1>

<H4>Extract Data</H4>

In [16]:
import pandas as pd 

licensing_csv_url = '/Users/leslieleiva/Documents/GitHub/California-Medical-Board/Resources/LIcensing_performance_data_2022.csv'
disciplinary_xlsx_url = '/Users/leslieleiva/Documents/GitHub/California-Medical-Board/Resources/Disciplinary-Alert-Actions-2022.xlsx'

licensing_df = pd.DataFrame(pd.read_csv(licensing_csv_url))
disciplinary_df = pd. DataFrame(pd.read_excel(disciplinary_xlsx_url))

licensing_df.head()

,Fiscal Year,Year,Quarter,Month,Date,DCA Entity,License Type Code,License Type,Transaction Code,Application Type,Target Cycle Time Complete Applications,Volume Complete Applications,Cycle Time Complete Applications,Volume Incomplete Applications,Cycle Time Incomplete Applications,Volume Renewal Applications,Cycle Time Renewal Applications,Comment,Website
0,6/30/2022,2022,Q2,December,12/31/2021,Acupuncture Board,NaN,Licensed Acupuncturist,NaN,Initial License Application,9,27.0,3.0,0.0,0,0,0.0,NaN,NaN
1,6/30/2022,2022,Q2,December,12/31/2021,Acupuncture Board,NaN,Licensed Acupuncturist,NaN,Renewal Application,0,0.0,0.0,0.0,0,479,3.0,NaN,NaN
2,6/30/2022,2022,Q2,November,11/30/2021,Acupuncture Board,NaN,Licensed Acupuncturist,NaN,Initial License Application,9,20.0,3.0,0.0,0,0,0.0,NaN,NaN
3,6/30/2022,2022,Q2,November,11/30/2021,Acupuncture Board,NaN,Licensed Acupuncturist,NaN,Renewal Application,0,0.0,0.0,0.0,0,375,3.0,NaN,NaN
4,6/30/2022,2022,Q2,October,10/31/2021,Acupuncture Board,NaN,Licensed Acupuncturist,NaN,Initial License Application,9,27.0,3.0,0.0,0,0,0.0,NaN,NaN


In [17]:
disciplinary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ALERT 
ISSUE DATE  838 non-null    datetime64[ns]
 1   NAME               838 non-null    object        
 2   CITY/STATE         838 non-null    object        
 3   LICENSE
NUMBER     838 non-null    object        
 4   TYPE OF ACTION     838 non-null    object        
 5   DATE
OF ACTION     838 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(4)
memory usage: 39.4+ KB


<H4>Extract/Read SQL Database</H4>

In [18]:
#Import dependencies
import sqlite3

# Path to your SQLite database
db_file = '/Users/leslieleiva/Documents/GitHub/California-Medical-Board/Resources/MBC_Physician_Surgeon_Database.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# List of tables to read from the database
tables_list = [
    'AdministrativeDisciplinaryAction',
    'FelonyConviction',
    'HospitalDisciplinaryAction',
    'License',
    'MisdemeanorConviction',
    'REF_SchoolCode'
]

# Dictionary to hold the DataFrames
dataframes = {}

for table in tables_list:
    # Query the table and load into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    dataframes[table] = df

# Close the connection
conn.close()

# Now you can access each DataFrame using the table name as the key
for table_name, df in dataframes.items():
    print(f"\nData from table: {table_name}")
    print(df.head())  # Display the first few rows of each DataFrame



Data from table: AdministrativeDisciplinaryAction
  LicenseID     CaseNumber            Description        EffectiveDate
0    807802  16 1991013768   SURRENDER OF LICENSE  1994-05-31 00:00:00
1   2088873  16 2010206989  SURRENDER OF LICENSE.  2011-03-16 00:00:00
2    297159  D1 1990002905  SURRENDER OF LICENSE.  2000-02-03 00:00:00
3    243565  17 1993029527               REVOKED.  1998-11-16 00:00:00
4    762568  02 2002140691   PROBATION COMPLETED.  2014-02-01 00:00:00

Data from table: FelonyConviction
  LicenseID                                DescriptionOfAction  \
0   1050066  FOUND GUILTY OF THREE COUNTS OF VIOLATING TITL...   
1    685433  PLED GUILTY TO THE SINGLE-COUNT INFORMATION, A...   
2    161220  PLED GUILTY TO 1 COUNT VIOL. OF TITLE 21, U.S....   
3    510141  1 CT 182(A)(1)-PC (CONSPIRACY TO COMMIT MURDER...   
4   1819438      PLED GUILTY TO 1 COUNT 487.1-PC (GRAND THEFT)   

  EffectiveDateOfAction                                         Court  \
0   2012-01-12 00:

In [20]:
#How to access dataframes in dictionary list
dataframes[tables_list[0]].head()

,LicenseID,CaseNumber,Description,EffectiveDate
0,807802,16 1991013768,SURRENDER OF LICENSE,1994-05-31 00:00:00
1,2088873,16 2010206989,SURRENDER OF LICENSE.,2011-03-16 00:00:00
2,297159,D1 1990002905,SURRENDER OF LICENSE.,2000-02-03 00:00:00
3,243565,17 1993029527,REVOKED.,1998-11-16 00:00:00
4,762568,02 2002140691,PROBATION COMPLETED.,2014-02-01 00:00:00


<H4>Clean-up</H4>

In [24]:
#Simplifying database by reducing the amount of columns
licensing_count_df = licensing_df[[
    'Date',
    'DCA Entity',
    'License Type'
]]

licensing_sum_df = licensing_df[[
        'Date',
        'DCA Entity',
        'License Type',
        'Volume Complete Applications',
        'Volume Incomplete Applications',
        'Volume Renewal Applications'
        
]]

#Group by date count
licensing_count_df = licensing_count_df.groupby('Date').count()

#Group by date sum
licensing_date_sum_df = licensing_sum_df.groupby('Date').sum()

#Delete columns
del licensing_date_sum_df['DCA Entity']
del licensing_date_sum_df['License Type']
del licensing_date_sum_df['Volume Renewal Applications']

#Group by License Type
licensing_type_sum_df = licensing_sum_df.groupby('License Type').sum()

#delete columns
del licensing_type_sum_df['Date']
del licensing_type_sum_df['DCA Entity']
del licensing_type_sum_df['Volume Renewal Applications']

#Create list of license types
license_type_list = list(licensing_type_sum_df.index)

# Path to the output file
output_file = "Output/List_Licenses.txt"

# Open the file in write mode
with open(output_file, 'w') as file:
    for item in license_type_list:
        file.write("%s\n" % item)  # Write each item followed by a newline character

In [106]:
import datetime as dt

#Delete Unneeded columns
DisciplinaryAction_df = dataframes[tables_list[0]][[
    'EffectiveDate',
    'Description',
]]

DisciplinaryAction_df = DisciplinaryAction_df.rename(columns={
    'EffectiveDate':'Date',
    'Description':'Disciplinary Action'
})

# # Convert 'Date' column to datetime
DisciplinaryAction_df['Date'] = pd.to_datetime(DisciplinaryAction_df['Date'],
                                               format='%Y-%m-%d %H:%M:%S',
                                               errors='coerce'
                                               )

# Filter for only the rows where the 'Date' column is in the year 2022
DisciplinaryAction_df_2022 = DisciplinaryAction_df[DisciplinaryAction_df['Date'].dt.year == 2022]

replacements = {
    'PROBATION COMPLETED':'PROBATION COMPLETED',
    'PROBATION COMPLETED': 'PROBATION COMPLETED.',
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS SCHEDULE II CONTROLLED SUBSTANCES, AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN CONCIERGE SERVICE PRACTICE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC SURGERY OR COSMETIC PROCEDURES OR UTILIZING INTRAVENOUS CONSCIOUS SEDATION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. AHMAD IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MCKENZIE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY OPIOID MEDICATIONS EXCEPT FOR BUPRENORPHINE, PROVIDED DR. DYNE MAINTAINS A VALID WAIVER AS A QUALIFIED PRACTITIONER. DURING PROBATION, DR. DYNE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY SCHEDULE II CONTROLLED SUBSTANCE. DURING PROBATION, DR. CAMHI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES; AND PROHIBITED FROM PRACTICING PSYCHIATRY.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SAAL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SAUNDERS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PALLAS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT PRACTICE COSMETIC MEDICINE UNTIL SUCESSFULLY COMPLETING THE CLINICAL COMPETENCE ASSSESSMENT PROGRAM. DURING PROBATION, DR. MILSTEIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. GUPTA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SELF-PRESCRIBING PRESCRIPTION DRUGS/MEDICATIONS; AND PROHIBTED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '5 YR PROBATION' : "FIVE YEARS' PROBATION TO BE SERVED CONCURRENTLY WITH AND UNDER THE SAME TERMS AND CONDITIONS IN CASE NUMBER 03-2013-234324, WITH ADDITIONAL TERMS AND CONDITIONS.  SHALL NOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION AND NOTFIIED BY THE BOARD OR ITS DESIGNEE THAT DR. APAYDIN IS FIT TO PRACTICE MEDICINE SAFELY (CONDITION MET).",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MACMURRAY IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. FRISOLI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. JAZAYERI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS; PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES, EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE V, UNTIL SUCCESSFUL COMPLETION OF A PRESCRIBING PRACTICES COURSE, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LE IS PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PREIKSAITIS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. HAKALA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PRESCRIBING SCHEDULE II CONTROLLED SUBSTANCES TO PATIENTS FOR CHRONIC PAIN MANAGEMENT THERAPY BUT CAN PRESCRIBE SCHEDULE II CONTROLLED SUBSTANCES TO TREAT ACUTE PAIN IN A QUANTITY THAT IS MEDICALLY CONSISTENT WITH A FIVE-DAY PRESCRIPTION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NGUYEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM WRITING, ISSUING OR OTHERWISE AIDING AND ABETTING THE WRITING OR ISSUANCE OF EXEMPTIONS FROM ANY VACCINE FOR ANY PATIENT OR OTHER PERSON; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL HAVE A THIRD-PARTY CHAPERONE PRESENT WHILE CONSULTING, EXAMINING OR TREATING FEMALE PATIENTS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BABAALI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PERFORMING STRABISMUS SURGERIES; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULES III, IV, AND V (THIS RESTRICTION SHALL NOT APPLY TO CONTROLLED SUBSTANCES ADMINISTERED TO PATIENTS WHO ARE ADMITTED TO THE HOSPITAL AND ONLY FOR THEIR IMMEDIATE USE WHILE THEY REMAIN SO ADMITTED), AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSES OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC PROCEDURES WITH A FRACTIONATED CO2 LASER AND FROM DELEGATING THE USE OF A FRACTIONATED CO2 LASER OR A MULTIWAVE LOCKED SYSTEM (MLS) LASER; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. POLAK IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PROVIDING CARE AND TREATMENT TO PATIENTS UNDER SIXTY (60) DAYS OF AGE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS' PROBATION TO RUN CONCURRENT, BUT SEPARATE AND APART FROM, THE PROBATIONARY ORDER IN CASE NO. 800-2015-016568, WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. POLIQUIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. YAREMA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. RABIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. IBRAHIM IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEAUCLAIR IS PROHIBITED FROM SUPERVISING ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PERSAUD IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS AND A 30 DAY ACTUAL SUSPENSION FROM THE PRACTICE OF MEDICINE TO BE COMPLETED BETWEEN JANUARY 1, 2023, AND JANUARY 1, 2024. DURING PROBATION, DR. PURCELL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING ANY AND ALL COSMETIC AND/OR AESTHETIC SURGICAL PROCEDURES OF ANY KIND, ON ANY PATIENT, AT ANY LOCATION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. ROSENBERG IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KHAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC LASER PROCEDURES EXCEPT FOR COSMETIC LASER HAIR REMOVAL, PROHIBITED FROM SUPERVISING ANYONE PERFORMING COSMETIC LASER PROCEDURES AND/OR PROHIBITED FROM DIRECTLY PROFITING IN ANY WAY FROM THE PERFORMANCE OF COSMETIC LASER PROCEDURES BEING DONE BY DR. GOODWIN OR SOMEONE UNDER HER SUPERVISION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KIRZNER IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MARIANO IS PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES, EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE IV AND V, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LEE IS PROHBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA UNTIL SUCCESSFUL COMPLETION OF A CLINICAL COMPETENCE ASSESSMENT PROGRAM; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PROVIDING TREATMENT TO DR. KIM?S FAMILY MEMBERS OR EMPLOYEES, EXCEPT TO DR. KIM?S FATHER IN EMERGENCY SITUATIONS ONLY  ; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS OR ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LIPSHITZ IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LARDON IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NIELSEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BHANDARI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEYRER IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MCCALLION IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KLASSEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. WILLIAMS IS PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM WRITING, ISSUING OR OTHERWISE AIDING AND ABETTING THE WRITING OR ISSUANCE OF EXEMPTIONS FROM ANY VACCINE FOR ANY PATIENT OR OTHER PERSONS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM HOLDING ANY OWNERSHIP INTEREST IN A MEDICAL OFFICE OR CLINIC; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE IV AND V, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NGUYEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. ROSETE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEESLEY IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. TOUSARKISSIAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS  LISTED IN SCHEDULE V UNTIL COMPLETION OF A PRESCRIBING PRACTICES COURSE (CONDITION SATISFIED), AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PERFORMING ANY VASCULAR PROCEDURE THAT INVOLVES THE USE OF CONTRAST DYE ON A PATIENT WITH A HISTORY OF RENAL FAILURE OR RENAL INSUFFICIENCY WITHOUT A PRIOR CONSULTATION WITH A NEPHROLOGY SPECIALIST; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. DHALIWAL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. HODGSON IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DR. MALEK IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NASIR IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LEE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: CANNOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES UNTIL COMPLETION OF A PRESCRIBING PRACTICES COURSE; SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA. PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM UTILIZING VASER TECHNOLOGY WHILE PERFORMING LIPOSUCTION PROCEDURES; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ISSUING, RENEWING, OR EXTENDING ANY MEDICAL VACCINE EXEMPTIONS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS' PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: CANNOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES UNTIL COMPLETING A PRESCRIBING PRACTICES COURSE (CONDITION SATISFIED); AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. DOBKIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '3 YR PROBATION' : "EFFECTIVE 09/04/2020, CASE NO. 800-2017-032412: THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM MAKING OR ISSUING ANY WRITTEN EXCEPTION FROM IMMUNIZATION, OR ANY OTHER WRITTEN STATEMENTS PROVIDING THAT ANY CHILD IS EXEMPT FROM THE REQUIREMENTS OF CHAPTER 1 COMMENCING WITH SECTIONS 120325, 120400, 120405, 120410, AND 120415 OF THE HEALTH AND SAFETY CODE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES. EFFECTIVE 01/21/2022, NEW DECISION SUPERSEDES THE PROBATIONARY ORDER IN CASE NO. 800-2017-032412; FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULES III, IV AND V; AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT. RESTRICTIONS: PROHIBITED FROM MAKING OR ISSUING ANY WRITTEN EXEMPTION FROM IMMUNIZATION, OR ANY OTHER WRITTEN STATEMENTS PROVIDING THAT ANY CHILD IS EXEMPT FROM THE REQUIREMENTS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MAMO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. EFFECTIVE 09/23/2022, CASE NO. 8002018047615 WILL RUN CONSECUTIVE TO AND SUPERSEDE THE TERMS OF PROBATION IN CASE NO. 8002017031593. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LAO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES; PROHIBITED FROM ENGAGING IN THE PRACTICE OF MEDICINE IN DR. SAIFY?S OR PATIENT?S PLACE OF RESIDENCE, UNLESS THE PATIENT RESIDES IN A SKILLED NURSING FACILITY OR OTHER SIMILAR LICENSED FACILITY.",
    '2 YR PROBATION' : "TWO YEARS' PROBATION WITH VARIOUS TERMS AND CONDITIONS, AND SHALL RUN CONSECUTIVE TO, AND SHALL TAKE EFFECT IMMEDIATELY UPON COMPLETION OF THE PROBATIONARY ORDER IN CASE NO. 800-2016-022245. RESTRICTIONS:  PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS, AND A SIXTY (60) DAY SUSPENSION. SUSPENSION BEGINS 04/09/2022 UNTIL 06/07/2022 (CONDITION SATISFIED). CANNOT PRACTICE MEDICINE UNTIL COMPLETING A PSYCHIATRIC EVALUATION.(CONDITION SATISFIED) DURING PROBATION, DR. HUTCHMAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. CANNOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION (CONDITION SATISFIED). SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ACCEPTING ANY NEW FEMALE PATIENTS FOR CARE AND TREATMENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: SHALL NOT PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES; SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR PERSONAL MEDICAL PURPOSES; SHALL NOT PRACTICE UNTIL PROVIDING DOCUMENTARY PROOF TO THE BOARD THAT DEA PERMIT HAS BEEN SURRENDERED FOR CANCELLATION; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '7 YR PROBATION' : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PERFORMING ARTERIAL OR VENOUS STENTING, EXCEPT UNDER DIRECT SUPERVISION BY ANOTHER PHYSICIAN OR WITHIN THE COURSE OF A FORMAL TRAINING PROGRAM; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    'PUBLIC REPRIMAND' : "PUBLIC REPRIMAND WILL BE ISSUED UPON COMPLETION OF TERMS.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PATINO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SUMARTO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. CHANG IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS OR ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '3 YR PROBATION' : "EFFECTIVE 08/18/2017, CASE NO. 09-2013-230441: THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MARCINKUS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES. EFFECTIVE 05/05/2022, NEW DECISION EXTENDS PROBATION SEVEN YEARS IN CASE NO. 09-2013-230441 WITH VARIOUS TERMS AND CONDITIONS, AND AN ACTUAL SUSPENSION FOR NINETY (90) DAYS. SUSPENSION BEGINS 05/21/2022 UNTIL 08/18/2022.",
    '4 YR PROBATION' : "EFFECTIVE 07/17/2020, CASE NO. 800-2016-029031: FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. EFFECTIVE 02/04/2022, NEW DECISION: EIGHT YEARS PROBATION TO RUN CONCURRENT WITH THE DISCIPLINARY ORDER IN CASE NO. 800-2016-029031, WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION AND NOTIFIED BY THE BOARD OR ITS DESIGNEE THAT DR. PARSAPOUR IS FIT TO PRACTICE MEDICINE SAFELY. RESTRICTIONS: PROHIBITED FROM PERFORMING SERVICES RELATING TO THE RECOMMENDATION OF MARIJUANA AND FROM RECOMMENDING MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "EFFECTIVE 08/23/2013, CASE NO. 04 2007-181659: SEVEN YEARS PROBATION WITH ACTUAL SUSPENSION. SUSPENSION SERVED 08/23/13 UNTIL 01/21/14. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE AND SUPERVISING PHYSICIAN ASSISTANTS. EFFECTIVE 04/08/2022, NEW DECISION SUPERSEDES ALL OTHER TERMS AND CONDITIONS IN CASE NO. 04-2007-181659. FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM EVALUATING ANY PATIENT FOR, OR ISSUING, ANY EMOTIONAL SUPPORT ANIMAL OR MEDICAL CANNABIS RECOMMENDATION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MAGIMBI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITONS. DURING PROBATION, DR. RINE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. CANNOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION (CONDITION SATISFIED). DURING PROBATION, DR. DESAI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING ANY PROCEDURES REQUIRING ANY FORM OF SEDATION IN A NON-ACCREDITED SURGERY CENTER; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '8 YR PROBATION' : "EFFECTIVE 10/18/2018, CASE NO. 800-2017-038264: EIGHT YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE, SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES. EFFECTIVE 03/11/2022, NEW DECISION EXTENDS PROBATION ONE ADDITIONAL YEAR WITH THE SAME TERMS AND CONDITIONS AS THE CURRENT PROBATION.",
    '10 YR PROBATION' : "TEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS AND A 90 DAY SUSPENSION. SUSPENSION BEGINS ON 10/07/2022 UNTIL 01/05/2023. RESTRICTIONS: PROHIBITED FROM PERFORMING AND/OR PARTICIPATING IN VASER LIPOSUCTION SURGERIES/PROCEDURES UNTIL COMPLETION OF A CLINICAL COMPETENCE ASSESSMENT PROGRAM (CONDITION SATISFIED); AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    'ACCUSATION FILED' : "ACCUSATION FILED. THE PHYSICIAN HAS NOT HAD A HEARING OR BEEN FOUND GUILTY OF ANY CHARGES.",
    'ACCUSATION FILED' : "AMENDED ACCUSATION FILED. THE PHYSICIAN HAS NOT HAD A HEARING OR BEEN FOUND GUILTY OF ANY CHARGES.",
    'TEMPORARY CEASE PRACTICE' : "CANNOT PRACTICE MEDICINE UNTIL COMPLETION OF A CLINICAL COMPETENCE ASSESSMENT PROGRAM, A CLINICAL DIAGNOSTIC EVALUATION, A PSYCHIATRIC EVALUATION, AND A MEDICAL EVALUATION, AND OBTAINING MEDICAL TREATMENT (CONDITION MET). SEVEN YEARS' PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    'AUTOMATIC SUSPENSION' : "AUTOMATIC SUSPENSION ORDER ISSUED-NO PRACTICE, BY OPERATION OF LAW, THE EFFECTIVE DATE OF THE ORDER IS 12/09/2022.",
    'CEASE PRACTICE' : "CEASE PRACTICE ORDER ISSUED - NO PRACTICE ALLOWED.",
    'CEASE PRACTICE' : "CEASE PRACTICE ORDER ISSUED-NO PRACTICE.",
    '3 YR PROBATION' : "EFFECTIVE 09/04/2020, CASE NO. 800-2017-032412: THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM MAKING OR ISSUING ANY WRITTEN EXCEPTION FROM IMMUNIZATION, OR ANY OTHER WRITTEN STATEMENTS PROVIDING THAT ANY CHILD IS EXEMPT FROM THE REQUIREMENTS OF CHAPTER 1 COMMENCING WITH SECTIONS 120325, 120400, 120405, 120410, AND 120415 OF THE HEALTH AND SAFETY CODE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES. EFFECTIVE 01/21/2022, NEW DECISION SUPERSEDES THE PROBATIONARY ORDER IN CASE NO. 800-2017-032412; FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULES III, IV AND V; AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT. RESTRICTIONS: PROHIBITED FROM MAKING OR ISSUING ANY WRITTEN EXEMPTION FROM IMMUNIZATION, OR ANY OTHER WRITTEN STATEMENTS PROVIDING THAT ANY CHILD IS EXEMPT FROM THE REQUIREMENTS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '7 YR PROBATION' : "EXTENDS PROBATION FOR SEVEN YEARS WITH VARIOUS TERMS AND CONDITIONS.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. AHMAD IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BABAALI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. FRISOLI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. GUPTA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. HAKALA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. JAZAYERI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LE IS PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MACMURRAY IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MCKENZIE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NGUYEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PALLAS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PREIKSAITIS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SAAL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SAUNDERS IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN CONCIERGE SERVICE PRACTICE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PERFORMING STRABISMUS SURGERIES; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULES III, IV, AND V (THIS RESTRICTION SHALL NOT APPLY TO CONTROLLED SUBSTANCES ADMINISTERED TO PATIENTS WHO ARE ADMITTED TO THE HOSPITAL AND ONLY FOR THEIR IMMEDIATE USE WHILE THEY REMAIN SO ADMITTED), AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSES OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC PROCEDURES WITH A FRACTIONATED CO2 LASER AND FROM DELEGATING THE USE OF A FRACTIONATED CO2 LASER OR A MULTIWAVE LOCKED SYSTEM (MLS) LASER; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC SURGERY OR COSMETIC PROCEDURES OR UTILIZING INTRAVENOUS CONSCIOUS SEDATION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PRESCRIBING SCHEDULE II CONTROLLED SUBSTANCES TO PATIENTS FOR CHRONIC PAIN MANAGEMENT THERAPY BUT CAN PRESCRIBE SCHEDULE II CONTROLLED SUBSTANCES TO TREAT ACUTE PAIN IN A QUANTITY THAT IS MEDICALLY CONSISTENT WITH A FIVE-DAY PRESCRIPTION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SELF-PRESCRIBING PRESCRIPTION DRUGS/MEDICATIONS; AND PROHIBTED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES; AND PROHIBITED FROM PRACTICING PSYCHIATRY.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM WRITING, ISSUING OR OTHERWISE AIDING AND ABETTING THE WRITING OR ISSUANCE OF EXEMPTIONS FROM ANY VACCINE FOR ANY PATIENT OR OTHER PERSON; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS SCHEDULE II CONTROLLED SUBSTANCES, AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR THE PERSONAL MEDICAL PURPOSES OF THE PATIENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS; PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES, EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE V, UNTIL SUCCESSFUL COMPLETION OF A PRESCRIBING PRACTICES COURSE, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL HAVE A THIRD-PARTY CHAPERONE PRESENT WHILE CONSULTING, EXAMINING OR TREATING FEMALE PATIENTS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY OPIOID MEDICATIONS EXCEPT FOR BUPRENORPHINE, PROVIDED DR. DYNE MAINTAINS A VALID WAIVER AS A QUALIFIED PRACTITIONER. DURING PROBATION, DR. DYNE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT PRACTICE COSMETIC MEDICINE UNTIL SUCESSFULLY COMPLETING THE CLINICAL COMPETENCE ASSSESSMENT PROGRAM. DURING PROBATION, DR. MILSTEIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '5 YR PROBATION' : "FIVE YEARS' PROBATION TO BE SERVED CONCURRENTLY WITH AND UNDER THE SAME TERMS AND CONDITIONS IN CASE NUMBER 03-2013-234324, WITH ADDITIONAL TERMS AND CONDITIONS.  SHALL NOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION AND NOTFIIED BY THE BOARD OR ITS DESIGNEE THAT DR. APAYDIN IS FIT TO PRACTICE MEDICINE SAFELY (CONDITION MET).",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS AND A 30 DAY ACTUAL SUSPENSION FROM THE PRACTICE OF MEDICINE TO BE COMPLETED BETWEEN JANUARY 1, 2023, AND JANUARY 1, 2024. DURING PROBATION, DR. PURCELL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEAUCLAIR IS PROHIBITED FROM SUPERVISING ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. IBRAHIM IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KHAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KIRZNER IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MARIANO IS PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES, EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE IV AND V, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PERSAUD IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. POLAK IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. POLIQUIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. RABIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. ROSENBERG IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. YAREMA IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING ANY AND ALL COSMETIC AND/OR AESTHETIC SURGICAL PROCEDURES OF ANY KIND, ON ANY PATIENT, AT ANY LOCATION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PERFORMING COSMETIC LASER PROCEDURES EXCEPT FOR COSMETIC LASER HAIR REMOVAL, PROHIBITED FROM SUPERVISING ANYONE PERFORMING COSMETIC LASER PROCEDURES AND/OR PROHIBITED FROM DIRECTLY PROFITING IN ANY WAY FROM THE PERFORMANCE OF COSMETIC LASER PROCEDURES BEING DONE BY DR. GOODWIN OR SOMEONE UNDER HER SUPERVISION; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PROVIDING CARE AND TREATMENT TO PATIENTS UNDER SIXTY (60) DAYS OF AGE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '4 YR PROBATION' : "FOUR YEARS' PROBATION TO RUN CONCURRENT, BUT SEPARATE AND APART FROM, THE PROBATIONARY ORDER IN CASE NO. 800-2015-016568, WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    'PROBATION COMPLETED' : "PROBATION COMPLETED",
    'SUSPENSION' : "FULL INTERIM SUSPENSION ORDER ISSUED-NO PRACTICE.",
    'INCOMPLETE PROBATION (DEATH)' : "NON-COMPLETION OF PROBATION DUE TO PHYSICIAN'S DEATH.",
    'ADDITIONAL PROBATION' : "ONE YEAR ADDITIONAL PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. COLE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "PETITION FOR REINSTATEMENT" : "PETITION FOR REINSTATEMENT OF REVOKED CERTIFICATE GRANTED.",
    "PETITION FOR REINSTATEMENT" : "PETITION FOR REINSTATEMENT OF SURRENDERED CERTIFICATE GRANTED.",
    "3 YR PROBATION" : "PLACED ON A NEW TERM OF THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. SHALL NOT PRACTICE MEDICINE UNTIL DR. ROGERS HAS PASSED THE REQUIRED EXAMINATION AND HAS BEEN NOTIFIED BY THE BOARD OR ITS DESIGNEE IN WRITING. (CONDITION SATISFIED) RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE, SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "ADDITIONAL PROBATION" : "PLACED ON PROBATION FOR THE DURATION OF PROBATION IN CASE NO. 8002015013651.  ADDITIONALLY PROHIBITED FROM PERFORMING ANY CARE OR TREATMENT WITH PATIENTS INVOLVING THE USE, MANAGEMENT, OR ANY SURGICAL PROCEDURE RELATED TO INTRATHECAL PUMPS, OR ADVISING ANY MEDICAL PROVIDER ON THE CARE OR TREATMENT OF PATIENTS INVOLVING THE USE, MANAGEMENT, OR ANY SURGICAL PROCEDURE RELATED TO INTRATHECAL PUMPS, FOR THE DURATION OF DR. SMITH'S PROBATION.",
    "PUBLIC REPRIMAND" : "PUBLIC LETTER OF REPRIMAND.",
    "PUBLIC REPRIMAND" : "PUBLIC REPRIMAND.",
    "REVOKED" : "REVOKED.",
    "7 YR PROBATION" : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS, AND A SIXTY (60) DAY SUSPENSION. SUSPENSION BEGINS 04/09/2022 UNTIL 06/07/2022 (CONDITION SATISFIED). CANNOT PRACTICE MEDICINE UNTIL COMPLETING A PSYCHIATRIC EVALUATION.(CONDITION SATISFIED) DURING PROBATION, DR. HUTCHMAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "7 YR PROBATION" : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. CANNOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION (CONDITION SATISFIED). SHALL NOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES AND SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A PATIENT'S PRIMARY CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "7 YR PROBATION" : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ACCEPTING ANY NEW FEMALE PATIENTS FOR CARE AND TREATMENT; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "7 YR PROBATION" : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PERFORMING ARTERIAL OR VENOUS STENTING, EXCEPT UNDER DIRECT SUPERVISION BY ANOTHER PHYSICIAN OR WITHIN THE COURSE OF A FORMAL TRAINING PROGRAM; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    "7 YR PROBATION" : "SEVEN YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: SHALL NOT PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES; SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO A PATIENT OR A CAREGIVER FOR THE POSSESSION OR CULTIVATION OF MARIJUANA FOR PERSONAL MEDICAL PURPOSES; SHALL NOT PRACTICE UNTIL PROVIDING DOCUMENTARY PROOF TO THE BOARD THAT DEA PERMIT HAS BEEN SURRENDERED FOR CANCELLATION; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    "SURRENDER OF LICENSE" : "SURRENDER OF LICENSE.",
    'SUSPENSION' : "SUSPENDED-NO PRACTICE PURSUANT TO SECTION 2310(A) OF THE BUSINESS AND PROFESSIONS CODE. BASED ON SUSPENSION OR REVOCATION IN ANOTHER STATE",
    'SUSPENSION' : "SUSPENDED-NO PRACTICE PURSUANT TO SECTION 2310(A) OF THE BUSINESS AND PROFESSIONS CODE. BASED ON SUSPENSION OR REVOCATION IN ANOTHER STATE.",
    '35 MONTH PROBATION' : "THIRTY-FIVE (35) MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. YEH IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. CANNOT ENGAGE IN THE PRACTICE OF MEDICINE UNTIL COMPLETION OF THE CLINICAL DIAGNOSTIC EVALUATION (CONDITION SATISFIED). DURING PROBATION, DR. DESAI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. CHANG IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS OR ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MAGIMBI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. PATINO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. SUMARTO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '35 MONTH PROBATION' : "THIRTY-FIVE MONTHS PROBATION WITH VARIOUS TERMS AND CONDITONS. DURING PROBATION, DR. RINE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEESLEY IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BEYRER IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. BHANDARI IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. CHAPPELL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. DHALIWAL IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. DOBKIN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. HODGSON IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. KLASSEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LARDON IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LEE IS PROHBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LEE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LIPSHITZ IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MCCALLION IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NASIR IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NGUYEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. NIELSEN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. ROSETE IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. TOUSARKISSIAN IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. WILLIAMS IS PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: CANNOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES UNTIL COMPLETION OF A PRESCRIBING PRACTICES COURSE; SHALL NOT ISSUE AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA. PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM HOLDING ANY OWNERSHIP INTEREST IN A MEDICAL OFFICE OR CLINIC; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ISSUING, RENEWING, OR EXTENDING ANY MEDICAL VACCINE EXEMPTIONS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA UNTIL SUCCESSFUL COMPLETION OF A CLINICAL COMPETENCE ASSESSMENT PROGRAM; PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; PROHIBITED FROM PROVIDING TREATMENT TO DR. KIM?S FAMILY MEMBERS OR EMPLOYEES, EXCEPT TO DR. KIM?S FATHER IN EMERGENCY SITUATIONS ONLY  ; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS OR ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM ORDERING, PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS LISTED IN SCHEDULE IV AND V, AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM PRESCRIBING, DISPENSING, ADMINISTERING, FURNISHING, OR POSSESSING ANY CONTROLLED SUBSTANCES EXCEPT FOR THOSE DRUGS  LISTED IN SCHEDULE V UNTIL COMPLETION OF A PRESCRIBING PRACTICES COURSE (CONDITION SATISFIED), AND FROM ISSUING AN ORAL OR WRITTEN RECOMMENDATION OR APPROVAL TO POSSESS OR CULTIVATE MARIJUANA; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM UTILIZING VASER TECHNOLOGY WHILE PERFORMING LIPOSUCTION PROCEDURES; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM WRITING, ISSUING OR OTHERWISE AIDING AND ABETTING THE WRITING OR ISSUANCE OF EXEMPTIONS FROM ANY VACCINE FOR ANY PATIENT OR OTHER PERSONS; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '3 YR PROBATION' : "THREE YEARS' PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: CANNOT ORDER, PRESCRIBE, DISPENSE, ADMINISTER, FURNISH, OR POSSESS ANY CONTROLLED SUBSTANCES UNTIL COMPLETING A PRESCRIBING PRACTICES COURSE (CONDITION SATISFIED); AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. LAO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. DURING PROBATION, DR. MAMO IS PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. EFFECTIVE 09/23/2022, CASE NO. 8002018047615 WILL RUN CONSECUTIVE TO AND SUPERSEDE THE TERMS OF PROBATION IN CASE NO. 8002017031593. RESTRICTIONS: PROHIBITED FROM ENGAGING IN THE SOLO PRACTICE OF MEDICINE; AND PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES.",
    '2 YR PROBATION' : "TWO YEARS PROBATION WITH VARIOUS TERMS AND CONDITIONS. RESTRICTIONS: PROHIBITED FROM SUPERVISING PHYSICIAN ASSISTANTS AND ADVANCED PRACTICE NURSES; PROHIBITED FROM ENGAGING IN THE PRACTICE OF MEDICINE IN DR. SAIFY?S OR PATIENT?S PLACE OF RESIDENCE, UNLESS THE PATIENT RESIDES IN A SKILLED NURSING FACILITY OR OTHER SIMILAR LICENSED FACILITY."
    }

replacements_reversed = {value: key for key, value in replacements.items()}

len(replacements_reversed)

# replacements_df = pd.DataFrame(replacements_reversed.items(),
#                                columns=[
#                                    'Disciplinary Action',
#                                    'Conviction'
#                                    ])



# DisciplinaryAction_df_2022_1 = DisciplinaryAction_df_2022.merge(replacements_df, on='Disciplinary Action', how='outer',indicator=False)

# replacements_df.info()
# DisciplinaryAction_df_2022_1.info()
# DisciplinaryAction_df_2022_1.to_csv('Output/DisciplinaryAction.csv')

20

In [ ]:
#DisciplinaryAction_df_2022 = DisciplinaryAction_df_2022.replace(replacements)


disciplinary_list = list(DisciplinaryAction_df_2022['Disciplinary Action'].unique())


# Path to the output file
output_file = "Output/List_DisciplinaryAction.txt"

# Open the file in write mode
with open(output_file, 'w') as file:
    for item in disciplinary_list:
        file.write("%s\n" % item)  # Write each item followed by a newline character

#DisciplinaryAction_df_2022
DisciplinaryAction_df_group = DisciplinaryAction_df_2022.groupby(['Disciplinary Action']).count()

DisciplinaryAction_df_group